In [5]:
# encoding=utf-8

import pandas as pd
import numpy as np
import importlib
import AnswerUtils
from collections import Counter
from sklearn import preprocessing
import re
import time

from AnswerUtils import CodeQA

In [2]:
codeQA = CodeQA()
code_keys = codeQA.get_keys()

100%|███████████████████████████████████████████████████████████████████████████████| 957/957 [00:02<00:00, 328.55it/s]


(40, 'f80687bc5ee67573e0f4f7596c5f0c5679688367', 1500084789000, 'Guuideal')
(41, 'b16b88777fc5576bf6bd34815d8aa8ed84b08837', 1500880817000, 'Guuideal')
(18, 'f600d0b4f4af19f3ef88a705fdd0f684c69560b8', 1486098813000, 'LaoHei')
(19, '95d163194eb307735e84809c9c541efe47e0039c', 1486098863000, 'LaoHei')
(12, '2073683e6ba357ae3c4220173fdd884242b8468b', 1500263557000, 'LeeSeulbi')
(13, '95cf40d56bb8995c41e4d4b7340cd756d0a481dc', 1500263678000, 'LeeSeulbi')
(80, '0486c8c577b882dd47415e5805a0ab943587d650', 1489590851000, 'Snowden')
(81, '81d1391e73dad2a4f019ce08bab7f2c26d006396', 1489590902000, 'Snowden')
(68, 'e696f16c8d7bca7a2af0da4cc1744749c030a067', 1487763119000, 'Totoro_jin')
(69, '9ccc3876bba3881989bba4fdc4dca92c27de64d0', 1487764681000, 'Totoro_jin')
(69, '9ccc3876bba3881989bba4fdc4dca92c27de64d0', 1487764681000, 'Totoro_jin')
(70, '2240cacbfadde93f477f5192dc945884d17a45e6', 1487764902000, 'Totoro_jin')
(70, '2240cacbfadde93f477f5192dc945884d17a45e6', 1487764902000, 'Totoro_jin')
(71, '

In [22]:
list(range(len(code_keys)))[4:]

[4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35]

In [ ]:
2,4,6,9,12,13,20,23,26,27,28,33,34

In [37]:
for i in list(range(len(code_keys)))[35:]:
    print(i, 100*i/len(code_keys), "%")
    sample_key = codeQA.get_key_by_index(i)
    test_info_pd = codeQA.get_testinfo_by_testid(sample_key)

    # Assert
    # compileErrStack_list = test_info_pd.drop_duplicates(['compileErrStack'])[['compileErrStack']].values
    # assert compileErrStack_list[0][0] != compileErrStack_list[0][0]
    assert len(Counter(test_info_pd.compileErrStack)) == 1
    assert len(Counter(test_info_pd.duration)) == 1
    owner_id = list(test_info_pd.columns).index("owner")
    tester_id = list(test_info_pd.columns).index("tester")
    tmp = test_info_pd.apply(lambda line: line[owner_id] == line[tester_id], axis=1).drop_duplicates()
    assert (len(tmp) == 1) and (tmp[0] == True)

    # Simple errorStack
    pattern = re.compile("\\tat com.tianmaying.*\(.*\)\\n")
    def get_most_value_info(row):
        searchobj = pattern.search(str(row))
        if searchobj:
            return searchobj.group().strip()
        else:
            return ''
    test_info_pd['errorStackSimple'] = test_info_pd.errStack.apply(get_most_value_info)

    # Label Encoder
    lbl = preprocessing.LabelEncoder()
    test_info_pd['titleid'] = lbl.fit_transform(test_info_pd.title)
    test_info_pd['resultid'] = lbl.fit_transform(test_info_pd.result)
    # test_info_pd['statusid'] = lbl.fit_transform(test_info_pd.status)
    test_info_pd['errorStackID'] = lbl.fit_transform(test_info_pd.errorStackSimple)

    # Status_str Encoder
    title_set = set(test_info_pd.titleid)
    status_str_list = []
    for test_id in test_info_pd.testId:
        status_str = ''
        for title_id in title_set:
            tmp = test_info_pd[(test_info_pd.testId == test_id) & (test_info_pd.titleid == title_id)].resultid.values[0]
            status_str += str(tmp)
        status_str_list.append(status_str)
    assert len(status_str_list) == len(test_info_pd)
    test_info_pd['status_str'] = status_str_list

    # errorStack_str Encoder
    title_set = set(test_info_pd.titleid)
    errorStack_str_list = []
    for test_id in test_info_pd.testId:
        errorStack_str = ''
        for title_id in title_set:
            tmp = test_info_pd[(test_info_pd.testId == test_id) & (test_info_pd.titleid == title_id)].errorStackID.values[0]
            assert tmp < 100
            if tmp < 10: errorStack_str += '0' + str(tmp) + '-'
            else: errorStack_str += str(tmp) + '-'
        errorStack_str_list.append(errorStack_str)
    assert len(errorStack_str_list) == len(test_info_pd)
    test_info_pd['errorStack_str'] = errorStack_str_list

    # get Q&A
    test_reviews_pd = codeQA.get_reviews_by_key(sample_key)
    answer_df = codeQA.get_answers_by_reviews(test_reviews_pd)
    question_df = codeQA.get_questions_by_reviews(test_reviews_pd)
    more_status_pd = test_info_pd[['testId', 'errorStack_str', 'status_str']].drop_duplicates()
    prefix = sample_key['course'].values[0]+"+"+sample_key['lesson'].values[0]+"+"+sample_key['question'].values[0]

    questino_status_pd = question_df.merge(more_status_pd, on=['testId'])[['testId','status_str', 'errorStack_str','content']]
    questino_status_pd = questino_status_pd.sort_values(['status_str'])
    questino_status_pd.to_csv("./output/"+prefix+"+question.csv", index=False, encoding="utf-8")

    answer_status_pd = answer_df.merge(more_status_pd, on=['testId'])[['testId','status_str', 'errorStack_str','content']]
    answer_status_pd = answer_status_pd.sort_values(['status_str'])
    answer_status_pd.to_csv("./output/"+prefix+"+answer.csv", index=False, encoding="utf-8")


35 97.22222222222223 %


In [16]:
test_info_pd.title.drop_duplicates()

0       完善显示博客内容方法处理博客不存在的情况
1                   完善删除博客方法
2                 完善显示博客内容方法
3                 完善显示博客列表方法
1772     initializationError
Name: title, dtype: object

In [18]:
test_info_pd[test_info_pd.title == 'initializationError']

,testId,title,result,duration,errMessage,errStack,tester,course,lesson,question,status,createdTime,compileErrStack,owner,id,errorStackSimple,titleid,resultid,errorStackID
1772,4c6f3c29-318c-4805-af4b-fc4833e54253,initializationError,failure,0,com/tianmaying/Post\n,java.lang.NoClassDefFoundError: com/tianmaying...,smartdavid,java-basic,blog-sample,default,failure,1488527803000,NaN,smartdavid,4c6f3c29-318c-4805-af4b-fc4833e54253,at com.tianmaying.TmyJunitRunner.runMain(TmyJu...,0,0,16
2201,cf15cee7-a9f8-46b8-b81a-ecfbef51a955,initializationError,failure,0,com/tianmaying/Post\n,java.lang.NoClassDefFoundError: com/tianmaying...,smartdavid,java-basic,blog-sample,default,failure,1488521079000,NaN,smartdavid,cf15cee7-a9f8-46b8-b81a-ecfbef51a955,at com.tianmaying.TmyJunitRunner.runMain(TmyJu...,0,0,16
2538,bc475d35-bfef-489e-b10b-330db21d076d,initializationError,failure,0,com/tianmaying/Post,java.lang.NoClassDefFoundError: com/tianmaying...,vailydia,java-basic,blog-sample,default,failure,1482217868000,NaN,vailydia,bc475d35-bfef-489e-b10b-330db21d076d,at com.tianmaying.TmyJunitRunner.runMain(TmyJu...,0,0,16
2551,bda2a533-53f9-4f09-b769-14dcc0129cb1,initializationError,failure,0,com/tianmaying/Post,java.lang.NoClassDefFoundError: com/tianmaying...,vailydia,java-basic,blog-sample,default,failure,1482217873000,NaN,vailydia,bda2a533-53f9-4f09-b769-14dcc0129cb1,at com.tianmaying.TmyJunitRunner.runMain(TmyJu...,0,0,16


In [13]:
set(test_info_pd.titleid)

{0, 1, 2, 3, 4}

In [10]:
test_info_pd[(test_info_pd.testId == test_id) ]

,testId,title,result,duration,errMessage,errStack,tester,course,lesson,question,status,createdTime,compileErrStack,owner,id,errorStackSimple,titleid,resultid,errorStackID
0,24645c26-5758-4bcc-bc7a-2c8044569c79,完善显示博客内容方法处理博客不存在的情况,passed,0,NaN,NaN,LaoHei,java-basic,blog-sample,default,failure,1489640454000,NaN,LaoHei,24645c26-5758-4bcc-bc7a-2c8044569c79,,3,1,0
1,24645c26-5758-4bcc-bc7a-2c8044569c79,完善删除博客方法,failure,0,输入的id与删除的博客id不一致\n,org.mockito.exceptions.verification.Verificati...,LaoHei,java-basic,blog-sample,default,failure,1489640454000,NaN,LaoHei,24645c26-5758-4bcc-bc7a-2c8044569c79,at com.tianmaying.SimpleTest.完善删除博客方法(SimpleTe...,1,0,4
2,24645c26-5758-4bcc-bc7a-2c8044569c79,完善显示博客内容方法,passed,0,NaN,NaN,LaoHei,java-basic,blog-sample,default,failure,1489640454000,NaN,LaoHei,24645c26-5758-4bcc-bc7a-2c8044569c79,,2,1,0
3,24645c26-5758-4bcc-bc7a-2c8044569c79,完善显示博客列表方法,passed,0,NaN,NaN,LaoHei,java-basic,blog-sample,default,failure,1489640454000,NaN,LaoHei,24645c26-5758-4bcc-bc7a-2c8044569c79,,4,1,0


In [12]:
test_info_pd[(test_info_pd.testId == '24645c26-5758-4bcc-bc7a-2c8044569c79') ]

,testId,title,result,duration,errMessage,errStack,tester,course,lesson,question,status,createdTime,compileErrStack,owner,id,errorStackSimple,titleid,resultid,errorStackID
0,24645c26-5758-4bcc-bc7a-2c8044569c79,完善显示博客内容方法处理博客不存在的情况,passed,0,NaN,NaN,LaoHei,java-basic,blog-sample,default,failure,1489640454000,NaN,LaoHei,24645c26-5758-4bcc-bc7a-2c8044569c79,,3,1,0
1,24645c26-5758-4bcc-bc7a-2c8044569c79,完善删除博客方法,failure,0,输入的id与删除的博客id不一致\n,org.mockito.exceptions.verification.Verificati...,LaoHei,java-basic,blog-sample,default,failure,1489640454000,NaN,LaoHei,24645c26-5758-4bcc-bc7a-2c8044569c79,at com.tianmaying.SimpleTest.完善删除博客方法(SimpleTe...,1,0,4
2,24645c26-5758-4bcc-bc7a-2c8044569c79,完善显示博客内容方法,passed,0,NaN,NaN,LaoHei,java-basic,blog-sample,default,failure,1489640454000,NaN,LaoHei,24645c26-5758-4bcc-bc7a-2c8044569c79,,2,1,0
3,24645c26-5758-4bcc-bc7a-2c8044569c79,完善显示博客列表方法,passed,0,NaN,NaN,LaoHei,java-basic,blog-sample,default,failure,1489640454000,NaN,LaoHei,24645c26-5758-4bcc-bc7a-2c8044569c79,,4,1,0


In [8]:
test_info_pd

,testId,title,result,duration,errMessage,errStack,tester,course,lesson,question,status,createdTime,compileErrStack,owner,id,errorStackSimple,titleid,resultid,errorStackID
0,24645c26-5758-4bcc-bc7a-2c8044569c79,完善显示博客内容方法处理博客不存在的情况,passed,0,NaN,NaN,LaoHei,java-basic,blog-sample,default,failure,1489640454000,NaN,LaoHei,24645c26-5758-4bcc-bc7a-2c8044569c79,,3,1,0
1,24645c26-5758-4bcc-bc7a-2c8044569c79,完善删除博客方法,failure,0,输入的id与删除的博客id不一致\n,org.mockito.exceptions.verification.Verificati...,LaoHei,java-basic,blog-sample,default,failure,1489640454000,NaN,LaoHei,24645c26-5758-4bcc-bc7a-2c8044569c79,at com.tianmaying.SimpleTest.完善删除博客方法(SimpleTe...,1,0,4
2,24645c26-5758-4bcc-bc7a-2c8044569c79,完善显示博客内容方法,passed,0,NaN,NaN,LaoHei,java-basic,blog-sample,default,failure,1489640454000,NaN,LaoHei,24645c26-5758-4bcc-bc7a-2c8044569c79,,2,1,0
3,24645c26-5758-4bcc-bc7a-2c8044569c79,完善显示博客列表方法,passed,0,NaN,NaN,LaoHei,java-basic,blog-sample,default,failure,1489640454000,NaN,LaoHei,24645c26-5758-4bcc-bc7a-2c8044569c79,,4,1,0
4,7ce64ba3-c828-4f00-a90a-3408f22bbbd7,完善显示博客列表方法,passed,0,NaN,NaN,picheng,java-basic,blog-sample,default,failure,1487776864000,NaN,picheng,7ce64ba3-c828-4f00-a90a-3408f22bbbd7,,4,1,0
5,7ce64ba3-c828-4f00-a90a-3408f22bbbd7,完善删除博客方法,passed,0,NaN,NaN,picheng,java-basic,blog-sample,default,failure,1487776864000,NaN,picheng,7ce64ba3-c828-4f00-a90a-3408f22bbbd7,,1,1,0
6,7ce64ba3-c828-4f00-a90a-3408f22bbbd7,完善显示博客内容方法处理博客不存在的情况,passed,0,NaN,NaN,picheng,java-basic,blog-sample,default,failure,1487776864000,NaN,picheng,7ce64ba3-c828-4f00-a90a-3408f22bbbd7,,3,1,0
7,7ce64ba3-c828-4f00-a90a-3408f22bbbd7,完善显示博客内容方法,failure,0,输入的id与获取的博客id不一致\n,org.mockito.exceptions.verification.Verificati...,picheng,java-basic,blog-sample,default,failure,1487776864000,NaN,picheng,7ce64ba3-c828-4f00-a90a-3408f22bbbd7,at com.tianmaying.SimpleTest.完善显示博客内容方法(Simple...,2,0,6
8,0827f6fb-23d6-472b-9250-de86efeecebe,完善显示博客列表方法,passed,0,NaN,NaN,picheng,java-basic,blog-sample,default,failure,1487811230000,NaN,picheng,0827f6fb-23d6-472b-9250-de86efeecebe,,4,1,0
9,0827f6fb-23d6-472b-9250-de86efeecebe,完善删除博客方法,passed,0,NaN,NaN,picheng,java-basic,blog-sample,default,failure,1487811230000,NaN,picheng,0827f6fb-23d6-472b-9250-de86efeecebe,,1,1,0
